In [1]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import psycopg2
import seaborn as sns
import matplotlib as plt

In [ ]:
conn = psycopg2.connect(
    dbname = 'salary_analysis_data_job_positions',
    user = 'postgres',
    password = 'Amir3525',
    host = 'localhost',
    port = '5432'
)

cur = conn.cursor()
print("Connected to the database")

Connected to the database


In [ ]:
cur.execute("SELECT * FROM salaries s LIMIT 10;")
rows = cur.fetchall()

for row in rows:
    print(rows)

# BASIC STATISTICS
### COUNTING TOTAL and DISTINCT job positions


In [7]:
cur.execute("SELECT COUNT(DISTINCT s.job_title) FROM salaries s;")
rows = cur.fetchall()

for row in rows:
    print("The number of different job positions is: ",rows)

The number of different job positions is:  [(312,)]


In [ ]:
cur.execute("SELECT s.job_title, COUNT(s.job_title) FROM salaries s GROUP BY s.job_title ORDER BY COUNT(s.job_title) DESC;")
rows = cur.fetchall()

for row in rows:
    print(row)

## Career Choice & Job Trends

### 1. Best Roles (part-time) for Beginners


In [ ]:
cur.execute("SELECT s.experience_level,	s.job_title, s.employment_type, ROUND(AVG(s.salary_in_usd),2) AS average_salary FROM salaries s WHERE s.experience_level = 'EN'	AND s.employment_type = 'PT' GROUP BY s.job_title, s.experience_level, s.employment_type ORDER BY AVG(s.salary_in_usd) DESC;")
rows = cur.fetchall()

for row in rows:
    print(row)

### 1.2. Best Roles (full-time) for Beginners


In [ ]:
cur.execute("SELECT s.experience_level,	s.job_title, s.employment_type, ROUND(AVG(s.salary_in_usd),2) AS average_salary FROM salaries s WHERE s.experience_level = 'EN' GROUP BY s.job_title, s.experience_level, s.employment_type ORDER BY AVG(s.salary_in_usd) DESC")
rows = cur.fetchall()

for row in rows:
    print(row)

## 2. Remote, Hybrid, or In-Person Preferences

In [ ]:
cur.execute("""
SELECT 
	COUNT(s.job_title) AS total_work_types,
	s.remote_ratio
FROM
	salaries s
GROUP BY
	s.remote_ratio
ORDER BY 
	COUNT(s.job_title) DESC
""")

rows = cur.fetchall()

for row in rows:
    print(row)

(69580, 0)
(18708, 100)
(296, 50)


## 2.2. Remote, Hybrid, or In-Person by company location
###  ONLINE

In [ ]:
cur.execute("""
SELECT 
    COUNT(job_title ) AS total_work_types,
    s.company_location,
    s.remote_ratio    
FROM
    salaries s
WHERE
	s.remote_ratio = '100'
GROUP BY
    s.remote_ratio,
    s.company_location
ORDER BY 
    total_work_types DESC
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 2.3. Remote, Hybrid, or In-Person by company location
###  HYBRID

In [ ]:
cur.execute("""
SELECT 
    COUNT(job_title ) AS total_work_types,
    s.company_location,
    s.remote_ratio    
FROM
    salaries s
WHERE
	s.remote_ratio = '50'
GROUP BY
    s.remote_ratio,
    s.company_location
ORDER BY 
    total_work_types DESC
LIMIT 10
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 2.4. Remote, Hybrid, or In-Person by company location
###  IN-PERSON

In [ ]:
cur.execute("""
SELECT 
    COUNT(job_title ) AS total_work_types,
    s.company_location,
    s.remote_ratio    
FROM
    salaries s
WHERE
	s.remote_ratio = '0'
GROUP BY
    s.remote_ratio,
    s.company_location
ORDER BY 
    total_work_types DESC
LIMIT 10
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 3. Career Growth Potential

In [ ]:
cur.execute("""
SELECT
	s.experience_level, 
	ROUND(AVG(s.salary_in_usd),2) AS average_salary,
	s.work_year
FROM
	salaries s
GROUP BY
	s.work_year,
	s.experience_level
ORDER BY
	s.work_year,
	CASE 
        WHEN s.experience_level = 'EN' THEN 1
        WHEN s.experience_level = 'MI' THEN 2
        WHEN s.experience_level = 'SE' THEN 3
        WHEN s.experience_level = 'EX' THEN 4
    END;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 3.2. Career Growth Potential
### Data Scientists

In [ ]:
cur.execute("""
SELECT
	s.experience_level, 
	ROUND(AVG(s.salary_in_usd),2) AS average_salary,
	s.work_year
FROM
	salaries s
WHERE
	s.job_title = 'Data Scientist'
GROUP BY
	s.work_year,
	s.experience_level
ORDER BY
	s.work_year,
	CASE 
        WHEN s.experience_level = 'EN' THEN 1
        WHEN s.experience_level = 'MI' THEN 2
        WHEN s.experience_level = 'SE' THEN 3
        WHEN s.experience_level = 'EX' THEN 4
    END;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 4. Comparing earnings across employment type and the most common employment type.

In [ ]:
cur.execute("""
SELECT
	s.employment_type,
	COUNT(s.employment_type) AS total_jobs_types,
	AVG(s.salary_in_usd) AS avg_salary_job_type
FROM
	salaries s
GROUP BY
	s.employment_type
ORDER BY
	CASE 
        WHEN s.employment_type = 'PT' THEN 1
        WHEN s.employment_type = 'FT' THEN 2
        WHEN s.employment_type = 'CT' THEN 3
        WHEN s.employment_type = 'FL' THEN 4
	END;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

## 4.2. Comparing earnings across employment_type and the most common employment type.
### Data Scientists

In [ ]:
cur.execute("""
SELECT
	s.employment_type,
	COUNT(s.employment_type) AS total_jobs_types,
	ROUND(AVG(s.salary_in_usd),2) AS avg_salary_job_type
FROM
	salaries s
WHERE 
	s.job_title = 'Data Scientist'
GROUP BY
	s.employment_type
ORDER BY
	CASE 
        WHEN s.employment_type = 'PT' THEN 1
        WHEN s.employment_type = 'FT' THEN 2
        WHEN s.employment_type = 'CT' THEN 3
        WHEN s.employment_type = 'FL' THEN 4
	END;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

In [ ]:
cur.close()
conn.close()
print("Connection closed")

Connection closed
